In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, label_oh_tf, ImageProcessor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import pickle
from fns4wandb import set_lossfn

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
model_vgg16 = vgg16(weights="IMAGENET1K_V1")
model_vgg16.classifier.pop(6)
model_vgg16.to(device)
#print(model_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [6]:
def preprocess_im(img_path):
    IP = ImageProcessor(device='cpu')
    img = cv2.imread(img_path) #
    img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    return img


In [7]:
# get len of 

file_path = r'/its/home/nn268/antvis/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))


#print(ids[4])
x, y = import_imagedata(file_path)

x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)


In [8]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [9]:
title = f'IDSW_VGG16_fine_80_112023'
save_dict = {'Run' : title,
            'Current_Epoch': 0}
                #r'/its/home/nn268/antvis/optics/
save_location = r'pickles/'#pickles

In [10]:
save_dict['Run']

'IDSW_VGG16_fine_80_112023'

In [11]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        return x

In [12]:
"""config = dict(
    epochs= 80, #30, 
    learning_rate =1e-6,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = (3,5),
    scheduler=0.2,
    f_lin_lay = 7168 #1024*7 = 7168
)"""

run_title = "IDSW_vgg_fine_hp_80_112023"

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'epochs':{
            'value': 100
        },

        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
        'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-7,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2, 0.3, 0.4, 0.6]
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['vgg16net_mlp']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/optics/AugmentedDS_IDSW/']
        }
        }
    }

sweep_id = wandb.sweep(config, project=f"{run_title}")

Create sweep with ID: 33b7fsm5
Sweep URL: https://wandb.ai/antvis/IDSW_vgg_fine_hp_80_112023/sweeps/33b7fsm5


In [13]:


def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, log='all', log_freq=10)
    
    loss_fn = set_lossfn(config.loss_fn) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
                                                                    #scheduler'
    ####
    
    #model = model.to(device)
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('E   ', epoch)
        t_correct = 0
        v_correct = 0
    
        if epoch == 0:
            #model = model.to('cpu')
            best_model = deepcopy(model)#.to(device)
            #model = model.to(device)
            
            
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx, img in enumerate(x_train): 
            model.train()
            x = preprocess_im(img)

            train_prediction = model.forward(x)
            
            y = label_oh_tf(y_train[idx], device=device, num_classes=11) # use same index val to index y (labels) and turn into onehot encoded label

            """
            if idx % 1000 == 0:
                print(idx, ' / ', len(x_train))
                !nvidia-smi
            """
            #print(train_prediction, train_label)
        
        
            t_loss = loss_fn(train_prediction, y)

            if train_prediction.argmax() == y.argmax():
                t_correct+=1

            
            optimizer.zero_grad()
            t_loss.backward()
            
            optimizer.step()
            scheduler.step()
            
            t_loss_list.append(t_loss.to('cpu'))
            t_predict_list.append(train_prediction.to('cpu'))
            t_label_list.append(y.to('cpu'))
            
            train_acc = (t_correct / len(x_train))
            t_accuracy_list.append(train_acc)
            
            
            
        print('validating...')
        
        for idx, img in enumerate(x_val):
            model.eval()
            x = preprocess_im(img)

            val_prediction = model.forward(x)
            
            y = label_oh_tf(y_val[idx], device=device, num_classes=11)
            """
            if idx % 100 == 0:
                print(idx, ' / ', len(x_val))
                !nvidia-smi
            """

            v_loss = loss_fn(val_prediction, y)
            
            if val_prediction.argmax() == y.argmax():
                v_correct +=1
                

            v_loss = v_loss.to('cpu')
            v_loss_list.append(v_loss.item())
            
            
            val_prediction = val_prediction.to('cpu')
            v_predict_list.append(val_prediction.detach().numpy())
            
            v_label_list.append(y.to('cpu').detach().numpy())
            
            val_acc = (v_correct / len(y_val))
            v_accuracy_list.append(val_acc)
            
        total_epochs += epoch

        if val_acc > best_acc:
    
            #print('Start Of SAve -----------------------------')
            #!nvidia-smi

            best_acc = val_acc
            
            #model = model.to('cpu')
            best_model = model#deepcopy(model)
            #model = model.to(device)
            

            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            
            model_architecture = [nn.Sequential(
                            model_vgg16,
                            Squeeze(),
                            nn.Linear(4096,11),
                            nn.Softmax(dim=0),
                        )]

            save_dict['model.state_dict'] = model.state_dict() #to('cpu').
            save_dict['model_architecture_untrained'] = model_architecture

            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        

        if (epoch+1)%2==0:
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model
    #labels = zip(t_label_list, v_label_list)
    #losses = zip(t_loss_list, v_loss_list)
    #predictions = zip(t_predict_list, v_predict_list)
    return model,save_dict



def pipeline(config): 
    
    vgg = nn.Sequential(
            model_vgg16,
            Squeeze(),
            nn.Linear(4096,11),
            nn.Softmax(dim=0),
        )
    
    #model_vgg16.to('cpu')
    
    vgg.to(device)
    
    #!nvidia-smi
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.MSELoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        model = vgg

        model, save_dict = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl

    return model, save_dict

In [ ]:
#model,save_dict = pipeline(config) #7,168

def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model, save_dict = pipeline(config)
        

wandb.agent(sweep_id, tr, count=20)

wandb: Agent Starting Run: 8j09dgeo with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	epochs: 100
wandb: 	first_lin_lay: 248832
wandb: 	image_path: /its/home/nn268/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 2.521455080168028e-05
wandb: 	loss_fn: MSE
wandb: 	model_name: vgg16net_mlp
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.6


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Widget Javascript not detected.  It may not be installed or enabled properly.


  0%|          | 0/100 [00:00<?, ?it/s]

E    0
training...
validating...


...

E    78
training...
validating...


 79%|███████▉  | 79/100 [2:06:07<33:43, 96.36s/it]

E    79
training...
validating...


 80%|████████  | 80/100 [3:19:10<7:40:47, 1382.35s/it]

E    80
training...


In [ ]:
!nvidia-smi

In [ ]:
# confusion matrix


def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()


In [ ]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

plot_confusion(t_predict, t_labels, 'Train Confusion Matrix')
plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [ ]:
model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(4096,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)

In [ ]:

vgg = nn.Sequential(
        model_vgg16,
        Squeeze(),
        nn.Linear(4096,11),
        nn.Softmax(dim=0),
    )

#model_vgg16.to('cpu')

vgg.to(device)
next(vgg.parameters()).is_cuda

In [ ]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

In [ ]:
model.to('cpu').state_dict()